# Imports

In [ ]:
from google.colab import drive
import os

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Load Data

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
os.chdir("drive/MyDrive/X/ML2 - Data Challenge/data")

In [ ]:
X = pd.read_csv('AMF_train_X.csv')
y = pd.read_csv('AMF_train_Y.csv')

In [ ]:
X.head()

,Index,Share,Day,Trader,OTR,OCR,OMR,min_time_two_events,mean_time_two_events,10_p_time_two_events,med_time_two_events,25_p_time_two_events,75_p_time_two_events,90_p_time_two_events,max_time_two_events,min_lifetime_cancel,mean_lifetime_cancel,10_p_lifetime_cancel,med_lifetime_cancel,25_p_lifetime_cancel,75_p_lifetime_cancel,90_p_lifetime_cancel,max_lifetime_cancel,NbTradeVenueMic,MaxNbTradesBySecond,MeanNbTradesBySecond,min_dt_TV1,mean_dt_TV1,med_dt_TV1,min_dt_TV1_TV2,mean_dt_TV1_TV2,med_dt_TV1_TV2,min_dt_TV1_TV3,mean_dt_TV1_TV3,med_dt_TV1_TV3,min_dt_TV1_TV4,mean_dt_TV1_TV4,med_dt_TV1_TV4,NbSecondWithAtLeatOneTrade
0,1,Isin_8,Date_24,Trader_10,2.272727,8.333333,12.500000,0.0,5117.830300,0.000000,419.688500,10.722543,984.320560,23151.83800,25139.598,682.153260,8272.770569,682.153260,984.32056,682.15326,23151.83800,23151.8380,23151.838,1,7,2.750000,351.702399,9639.216031,1585.079495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,2,Isin_87,Date_29,Trader_10,1.696629,25.166667,21.571429,0.0,1846.968401,0.000074,0.003374,0.000204,8.768699,2671.48830,31278.357,11.866026,3796.632686,11.866026,177.93991,73.74323,808.26230,21433.6840,21433.684,1,17,5.933333,0.000066,493.559196,0.000901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
2,3,Isin_87,Date_23,Trader_10,1.482759,47.300000,118.250000,0.0,686.300630,0.000071,0.000599,0.000129,5.725427,448.88360,30799.467,2.761036,1397.103679,2.761036,187.99548,19.77734,418.23984,1953.6235,10842.464,1,20,5.063492,0.000018,163.954785,0.000770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
3,4,Isin_12,Date_11,Trader_10,1.705882,14.500000,29.000000,0.0,2174.335265,0.000000,6.152666,0.000945,62.444176,19187.71900,19187.719,286.019320,9736.869034,286.019320,286.01932,286.01932,19187.71900,19187.7190,19187.719,1,8,4.250000,0.000148,1232.562133,0.004301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,Isin_87,Date_9,Trader_10,1.517730,26.750000,NaN,0.0,944.008551,0.000071,0.001364,0.000146,2.225420,273.45676,23164.514,2.798452,10066.803992,2.798452,1345.95280,662.57434,21903.78300,23164.5140,23164.514,1,19,3.710526,0.000025,248.073927,0.001364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38


In [ ]:
y.head()

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT


In [ ]:
y.shape

(86, 2)

# Preproc

In [ ]:
#other metohd
X_train = X[X['Trader'].isin(X['Trader'].unique()[:60])]
X_test = X[X['Trader'].isin(X['Trader'].unique()[61:])]
y_train = y[y['Trader'].isin(X['Trader'].unique()[:60])]
y_test = y[y['Trader'].isin(X['Trader'].unique()[61:])]

In [ ]:
X2 = X.set_index('Trader').drop(['Index', 'Share', 'Day'], axis=1)
y_new = X2.join(y.set_index('Trader'), on='Trader')['type']

In [ ]:
X2 = X2.fillna(0)

In [ ]:
X2.shape, y_new.shape

((105782, 35), (105782,))

In [ ]:
X2.head()

,OTR,OCR,OMR,min_time_two_events,mean_time_two_events,10_p_time_two_events,med_time_two_events,25_p_time_two_events,75_p_time_two_events,90_p_time_two_events,max_time_two_events,min_lifetime_cancel,mean_lifetime_cancel,10_p_lifetime_cancel,med_lifetime_cancel,25_p_lifetime_cancel,75_p_lifetime_cancel,90_p_lifetime_cancel,max_lifetime_cancel,NbTradeVenueMic,MaxNbTradesBySecond,MeanNbTradesBySecond,min_dt_TV1,mean_dt_TV1,med_dt_TV1,min_dt_TV1_TV2,mean_dt_TV1_TV2,med_dt_TV1_TV2,min_dt_TV1_TV3,mean_dt_TV1_TV3,med_dt_TV1_TV3,min_dt_TV1_TV4,mean_dt_TV1_TV4,med_dt_TV1_TV4,NbSecondWithAtLeatOneTrade
Trader,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trader_10,2.272727,8.333333,12.500000,0.0,5117.830300,0.000000,419.688500,10.722543,984.320560,23151.83800,25139.598,682.153260,8272.770569,682.153260,984.32056,682.15326,23151.83800,23151.8380,23151.838,1,7,2.750000,351.702399,9639.216031,1585.079495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
Trader_10,1.696629,25.166667,21.571429,0.0,1846.968401,0.000074,0.003374,0.000204,8.768699,2671.48830,31278.357,11.866026,3796.632686,11.866026,177.93991,73.74323,808.26230,21433.6840,21433.684,1,17,5.933333,0.000066,493.559196,0.000901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15
Trader_10,1.482759,47.300000,118.250000,0.0,686.300630,0.000071,0.000599,0.000129,5.725427,448.88360,30799.467,2.761036,1397.103679,2.761036,187.99548,19.77734,418.23984,1953.6235,10842.464,1,20,5.063492,0.000018,163.954785,0.000770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63
Trader_10,1.705882,14.500000,29.000000,0.0,2174.335265,0.000000,6.152666,0.000945,62.444176,19187.71900,19187.719,286.019320,9736.869034,286.019320,286.01932,286.01932,19187.71900,19187.7190,19187.719,1,8,4.250000,0.000148,1232.562133,0.004301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
Trader_10,1.517730,26.750000,0.000000,0.0,944.008551,0.000071,0.001364,0.000146,2.225420,273.45676,23164.514,2.798452,10066.803992,2.798452,1345.95280,662.57434,21903.78300,23164.5140,23164.514,1,19,3.710526,0.000025,248.073927,0.001364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y_new, test_size=0.33, random_state=42)

# Model

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
f1_score(y_test, y_pred, average='micro')

0.9777707754447277

# Reformat predictions

In [ ]:
len(X_test.index.unique())

83

In [ ]:
y_pred

array(['MIX', 'NON HFT', 'HFT', ..., 'MIX', 'MIX', 'HFT'], dtype=object)

In [ ]:
X_predicted = X_test.copy()

In [ ]:
X_predicted['prediction'] = y_pred

In [ ]:
X_predicted

,OTR,OCR,OMR,min_time_two_events,mean_time_two_events,10_p_time_two_events,med_time_two_events,25_p_time_two_events,75_p_time_two_events,90_p_time_two_events,max_time_two_events,min_lifetime_cancel,mean_lifetime_cancel,10_p_lifetime_cancel,med_lifetime_cancel,25_p_lifetime_cancel,75_p_lifetime_cancel,90_p_lifetime_cancel,max_lifetime_cancel,NbTradeVenueMic,MaxNbTradesBySecond,MeanNbTradesBySecond,min_dt_TV1,mean_dt_TV1,med_dt_TV1,min_dt_TV1_TV2,mean_dt_TV1_TV2,med_dt_TV1_TV2,min_dt_TV1_TV3,mean_dt_TV1_TV3,med_dt_TV1_TV3,min_dt_TV1_TV4,mean_dt_TV1_TV4,med_dt_TV1_TV4,NbSecondWithAtLeatOneTrade,prediction
Trader,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trader_233,2.350000,17.625000,23.500000,0.000000,97.044875,0.000006,9.875284,0.000011,80.751720,270.837070,1113.84270,1.804689,94.823033,1.804689,17.576826,4.676560,157.791170,329.641750,329.64175,11,23,1.777778,0.000020,269.329493,33.917540,0.000005,219.164803,45.315445,0.000133,319.409592,86.821270,0.000110,355.414881,74.874734,108,MIX
Trader_105,5.132075,27.200000,0.000000,0.000000,3938.277086,0.000000,0.000000,0.000000,1228.878000,16801.943000,44749.86300,10.121589,17575.052735,174.853530,16202.294000,1734.637800,25608.980000,41319.273000,44749.86300,1,7,1.292683,0.000059,347.204026,215.359653,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,82,NON HFT
Trader_226,5.839695,2.919847,0.000000,0.000000,923.659258,0.000000,124.255590,2.597940,489.207760,1869.357800,24463.01200,0.000031,1230.627581,2.842217,239.691350,44.403667,868.206050,2901.890000,24463.01200,5,4,1.204225,0.000002,169.184200,59.914820,0.000353,181.426969,67.987273,0.000353,178.920555,67.987273,0.000353,174.449030,63.360734,142,HFT
Trader_453,192.200000,4.037815,2.014675,0.000028,91.127473,1.800546,38.033417,11.346172,122.795680,238.646040,1048.31540,0.000054,274.040667,9.806944,195.160190,66.304760,362.153630,660.391360,1793.03170,2,11,1.375000,564.830712,925.978003,564.830712,564.830712,925.978003,564.830712,564.830712,925.978003,564.830712,564.830712,925.978003,564.830712,40,HFT
Trader_226,9.413643,2.544527,0.000000,0.000000,167.211959,0.000076,24.757190,1.351061,155.743040,437.646730,5595.42000,0.000036,199.470169,0.710668,44.307830,7.788269,211.758710,524.296000,5595.42000,7,18,1.621884,0.000001,25.810371,3.750141,0.000009,24.884210,11.154731,0.000009,24.696698,11.603815,0.000009,24.796140,11.154731,722,HFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trader_222,6.900000,2.464286,0.000000,0.000000,3220.670208,5.399853,887.159600,30.611053,3629.428200,9276.742000,9283.28600,24.133417,3680.573079,28.416615,3301.525000,61.151318,8580.507000,9278.916000,9283.28600,1,7,3.333333,0.000574,906.027927,15.344946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,NON HFT
Trader_209,17.217391,11.647059,1.528958,0.000000,875.425750,1.003573,137.055760,8.837206,530.524900,1210.176600,47896.20700,1.041189,4432.469072,5.654765,813.226900,90.194954,5368.976600,9837.434000,47896.20700,4,6,1.870968,0.000010,856.870264,315.021585,0.000234,715.828459,14.476697,0.000005,974.692374,14.476697,0.000488,657.270155,14.476697,31,MIX
Trader_233,3.424242,5.650000,42.375000,0.000000,134.486668,0.000007,66.626330,12.835981,237.209900,351.126130,615.79785,1.044497,221.555007,18.568832,190.105440,77.796970,349.211670,425.356170,615.79785,10,16,1.746606,0.000080,95.670386,34.736637,0.000002,112.180237,34.386590,0.000002,80.793269,19.715952,0.000088,79.920621,12.002482,221,MIX


In [ ]:
groups = X_predicted.groupby('Trader')['prediction'].apply(list)
df1 = groups.reset_index(name='prediction')

In [ ]:
df1

,Trader,prediction
0,Trader_10,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ..."
1,Trader_105,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ..."
2,Trader_107,"[MIX, MIX, MIX, MIX, MIX, MIX, MIX, MIX, MIX, ..."
3,Trader_110,[NON HFT]
4,Trader_114,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ..."
...,...,...
78,Trader_51,"[NON HFT, NON HFT, NON HFT, NON HFT, MIX, NON ..."
79,Trader_54,"[NON HFT, MIX, HFT, NON HFT, NON HFT, MIX, MIX..."
80,Trader_57,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT]"
81,Trader_59,"[MIX, NON HFT, NON HFT, NON HFT]"


In [ ]:
preds = df1['prediction']

In [ ]:
numerical_preds = [[t.count('NON HFT'), t.count('MIX'), t.count('HFT')] for t in preds]

In [ ]:
preds[2]

['MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'NON HFT',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'NON HFT',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',
 'MIX',


In [ ]:
numerical_preds[2]

[10, 602, 5]

In [ ]:
prop_mix = [p[1]/np.sum(p) for p in numerical_preds]
prop_hft = [p[2]/np.sum(p) for p in numerical_preds]

In [ ]:
prop_mix[2]

0.9756888168557536

In [ ]:
classes = []
for i in range(len(numerical_preds)):
  if prop_hft[i] >= 0.85:
    classes.append('HFT')
  elif prop_mix[i] >= 0.5:
    classes.append('MIX')
  else:
    classes.append('NON HFT')

In [ ]:
len(classes)

83

In [ ]:
trader_pred_final = df1.copy()

In [ ]:
trader_pred_final['class'] = classes

In [ ]:
trader_pred_final

,Trader,prediction,class
0,Trader_10,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ...",NON HFT
1,Trader_105,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ...",NON HFT
2,Trader_107,"[MIX, MIX, MIX, MIX, MIX, MIX, MIX, MIX, MIX, ...",MIX
3,Trader_110,[NON HFT],NON HFT
4,Trader_114,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT, ...",NON HFT
...,...,...,...
78,Trader_51,"[NON HFT, NON HFT, NON HFT, NON HFT, MIX, NON ...",NON HFT
79,Trader_54,"[NON HFT, MIX, HFT, NON HFT, NON HFT, MIX, MIX...",NON HFT
80,Trader_57,"[NON HFT, NON HFT, NON HFT, NON HFT, NON HFT]",NON HFT
81,Trader_59,"[MIX, NON HFT, NON HFT, NON HFT]",NON HFT


In [ ]:
true_values = y.join(trader_pred_final.set_index('Trader'), on='Trader')['type']
pred_classes = y.join(trader_pred_final.set_index('Trader'), on='Trader')['class']

In [ ]:
true_values

0         MIX
1     NON HFT
2     NON HFT
3     NON HFT
4     NON HFT
       ...   
81        MIX
82        MIX
83        MIX
84        HFT
85        MIX
Name: type, Length: 86, dtype: object

In [ ]:
pred_classes

0         MIX
1     NON HFT
2     NON HFT
3     NON HFT
4     NON HFT
       ...   
81        MIX
82        MIX
83        MIX
84        HFT
85        MIX
Name: class, Length: 86, dtype: object

In [ ]:
truth = np.array(true_values) == np.array(pred_classes)

In [ ]:
truth

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [ ]:
accuracy = np.sum(truth) / len(truth)
accuracy

0.9186046511627907

In [ ]:
f1_score(true_values, pred_classes, average='micro')

TypeError: ignored